# Topic Modeling demo (Arabic)

In [2]:
import numpy as np
import numpy.matlib
import pandas as pd
import lda
from glob import glob
from stop_words import get_stop_words
import os

## Loading data
The data we're using for this demo comes from the لنتفق الآن ("Let's Agree Now!") Facebook page. Most posts are in Arabic, and they will address a variety of different topics. Some posts come from the account owner, while the majority will be contributed by visitors to their page. The first five entries are shown below.

In [5]:
os.chdir('C:\\Users\\Craig\\Desktop\\Live projects\\Libya\\Python LDA')
xls = pd.concat([pd.read_excel(x) for x in glob('Lets Agree Now*.xls')])
pd.set_option('display.max_colwidth', 1000)
xls[['Author','Contents']].head(5)

,Author,Contents
0,Franc Slame,انتو تفرحو بناقلت نفط ونحن نفرح لتحري أرض الوطن ياانجاس
1,Salma Abu,وأخيرا
2,الحفره الحفره,سبب الخراب والدمار فبراير اليوم الاسود 2011/2/17
3,غدا اجمل,يا خوي اتق الله كيف تحلف بالله علي شي في علم الغيب .. الله وحده العالم بشن حيصير في البلاد .. ومن اللي حيرضى انه بلادنا تنباع ؟؟؟؟
4,Abdo Altwel,مفروض كل واحد فيهم ايروح على رجليه لن يوصل حوشه تو بعدين يتعلمو التشحيط


## Stop words
In the field of Natural Language Processing (NLP), certain words are known as *stop words*. These are very common words -- English examples could include "the", "of", "is", "and", "or", etc. -- that often don't tell us very much about the subject of the sentence that contains them. We'll be using a set of common Arabic stop words that are commonly used by NLP researchers. Here are the first 20, to give a sense of what kind of words we're talking about.

In [12]:
asw = get_stop_words('arabic')
print(asw[:20])

['فى', 'في', 'كل', 'لم', 'لن', 'له', 'من', 'هو', 'هي', 'قوة', 'كما', 'لها', 'منذ', 'وقد', 'ولا', 'نفسه', 'لقاء', 'مقابل', 'هناك', 'وقال']


If we need to, we can add more stopwords at this point

In [8]:
#print(asw[:-20:-1])
#asw.append('ليبيا') # Add "Libya"
#print(asw[:-20:-1])

['صفر', 'بها', 'اي', 'او', 'ان', 'اف', 'ثم', 'به', 'بن', 'حاليا', 'بشكل', 'غير', 'اجل', 'بان', 'اخرى', 'اربعة', 'اطار', 'صباح', 'شخصا']
['ليبيا', 'صفر', 'بها', 'اي', 'او', 'ان', 'اف', 'ثم', 'به', 'بن', 'حاليا', 'بشكل', 'غير', 'اجل', 'بان', 'اخرى', 'اربعة', 'اطار', 'صباح']


## Preparing for topic modeling
Topic modeling algorithms often struggle to identify the topic of very short pieces of text. We'll get around this by only paying attention to posts with more than 250 characters -- there are 2,342 such posts. We're also going to simplify things by removing numbers and URLs, since they're not likely to tell us much about topics.

Those long posts are then passed to an algorithm called a *vectorizer* that turns the set of posts into a matrix of numbers, because computers generally prefer to work with numbers. This is done by identifying the *vocabulary* of all words that appear at least once (45,085 of them) and counting the number of times that each word appears in each post. This means we have a 2342x45085 array of numbers (known as a *document-term matrix* or DTM), for 2,342 documents and 45,085 words.

In [10]:
xls['Filtered'] = xls['Contents'].replace(r'\d+','',regex=True)
xls['Filtered'] = xls['Filtered'].replace(r'https?://[\w./]+','',regex=True)

In [13]:
def getlen(x):
    try:
        return(len(x))
    except:
        return(0)
xls['strlen'] = xls['Filtered'].apply(getlen)
long_posts = xls[xls.strlen > 250].reset_index(drop=True)
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=asw) 
vect.fit(long_posts.Filtered)
long_dtm = vect.transform(long_posts.Filtered)
vocab = vect.get_feature_names()
long_dtm.shape

(2342, 45085)

## Fitting the topic model
The algorithm we're using is called LDA (Latent Dirichlet Allocation). The basic idea is that we give it a set of documents (or more precisely a DTM), and ask it to identify a specific number of topics (20 in this case). Each document is given a *probability* of belonging to each topic. For each topic, every word has a *weight* -- the words with the highest weights are the ones that are most important to the topic.

Part of the model-fitting algorithm requires a random-number generator -- this means that the results will be slightly different each time. We're going to fit the model twice, because we suspect that topics that are consistent between the two versions of the fitted model will be more reliable.

In [14]:
myLDA1 = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
myLDA1.fit(long_dtm)
myLDA2 = lda.LDA(n_topics=20, n_iter=1500, random_state=2)
myLDA2.fit(long_dtm)

In [15]:
tw1 = myLDA1.topic_word_
tw2 = myLDA2.topic_word_
ldamat1 = myLDA1.transform(long_dtm)
ldamat2 = myLDA2.transform(long_dtm)

## Summarizing topics
We can quickly get a sense of what some of the major topics on the "Let's Agree Now!" Facebook page are by looking at the 8 most heavily-weighted terms in each topic. Here are the results for the first version of the fitted model:

In [16]:
n_top_words = 8

def top_words(topic_dist,n_top_words=10):
    return(np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1])

for i, topic_dist in enumerate(tw1):
    topic_words = top_words(topic_dist,8)
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: الوطني الوفاق إن حكومة السيد رئيس مجلس أن
Topic 1: الآن لنتفق لنتفق_الآن type photos timeline الله تشجيع
Topic 2: الله الوكيل ونعم حسبي حسبنا العظيم سبحان الدولار
Topic 3: إلى أن المتحدة الليبي السياسي كوبلر حكومة الاتفاق
Topic 4: المجلس الرئاسي الوفاق الوطني لحكومة طرابلس رئيس السراج
Topic 5: علي الي الجيش بنغازي الغرب معمر الشعب العالم
Topic 6: أن إلى أو ورد وأن أي مجلس الدولة
Topic 7: الاتفاق السياسي للدولة الأعلى السويحلي المجلس الدولة حسب
Topic 8: فينا ﻣﻦ ﺑﺎﻟﻠﻪ ﺇﻻ ﺍﻟﻠﻪ منه ﻭﻻ ليبي
Topic 9: إلى أن داعش أي إن تنظيم بأن عبر
Topic 10: لله الله الملك مافي مابعده الآن لنتفق لنتفق_الآن
Topic 11: اللي مش البلاد علي بس والله الي يا
Topic 12: النواب مجلس أن المجلس الوفاق إلى الرئاسي الثقة
Topic 13: حكومة أن الليبي الحكومة الأوروبي الوفاق السراج الخارجية
Topic 14: وكالة التضامن أنباء إلى لنتفق الآن أن الليبية
Topic 15: الله الوطن يا علي والله نحن لنا ولكن
Topic 16: لنتفق الآن لنتفق_الآن libya com السلام فيديو visa
Topic 17: اللهم الله ال يا وا ﺍﻟﻠﻪ ﺍﻛﺒﺮ محمد
Topic 18: يجب نحن أو الب

To get a sense of how much we can trust the topics, we can look at the same output for the other version of the model and see which topics look similar.

In [17]:
for i, topic_dist in enumerate(tw2):
    topic_words = top_words(topic_dist,8)
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: المركزي مصرف الآن دولار دينار السيولة المصارف لنتفق
Topic 1: المجلس الرئاسي الوفاق لحكومة الوطني رئيس السراج طرابلس
Topic 2: ﻣﻦ ﺑﺎﻟﻠﻪ ﺇﻻ ﺍﻟﻠﻪ ﻭﻻ ﻗﻮ ﻋﻠﻰ ﺣﻮﻝ
Topic 3: الوطني الوفاق حكومة إن أن إلى السيد الليبي
Topic 4: النواب مجلس أن الوفاق المجلس إلى السياسي حكومة
Topic 5: الشعب الله الوطن نحن البلاد يجب هم ولكن
Topic 6: اللي مش علي البلاد بس والله ربي الي
Topic 7: أن علي أو الناس العالم هل لو نحن
Topic 8: المتحدة الليبي المؤتمر الاتفاق أن الأمم كوبلر السياسي
Topic 9: لنتفق الآن لنتفق_الآن type photos timeline أو الخير
Topic 10: أن إلى الخارجية الليبية الاتحاد الأوروبي داعش إن
Topic 11: الله لله الملك مافي مابعده خير الآن لنتفق
Topic 12: بنغازي الي علي انت انا الجيش كيف الغرب
Topic 13: وزارة أن محمد وزيرا زليتن العمل دولة الحكم
Topic 14: الآن لنتفق_الآن لنتفق type طرابلس مدينة بنغازي تشجيع
Topic 15: أن إلى الليبي أي الدولة الوطنية النفط طرابلس
Topic 16: ال اللي وا سيتم تعليقات لنتفق_الآن بـ لنتفق
Topic 17: الله يا والله الوطن لنا عليكم الحياة منا
Topic 18: أن التضامن أنباء وكال

## Finding matching topics
We know that only some topics will overlap between the two sets, and that the similar topics may be in a different order. We can easily measure the similarity between two topics by listing the top 100 words in each topic, and counting how many of those top words are in common between the two lists. If two topics have more than 50% of their top 100 words in common, we'll consider them to be a match.

In [18]:
def overlap(td1,td2,n=10):
    w1 = top_words(td1,n)
    w2 = top_words(td2,n)
    return(len(set(w1) & set(w2))/n)

match_pairs = []
n = tw1.shape[0]
for i in range(n):
    for j in range(n):
        o = overlap(tw1[i,:],tw2[j,:],100)
        if o > 0.5:
            match_pairs.append((i,j))
            
print(match_pairs)

[(0, 3), (1, 9), (3, 8), (4, 1), (11, 6), (12, 4), (19, 16)]


Out of our two sets of 20 topics, we have 12 matches. The numbers above mean that, for example, topic \#0 from the first model matches with \#3 from the second model, while topic \#1 from the first model matches with \#9 from the second.
## Example posts
Each post has a probability -- a number between 0 and 1 -- measuring how likely it is to fit in with any topic. To better understand what is in a topic, we can find a post with a very high probability for that topic. The example below is for topic \#4 in the first model, which (as far as I can tell from Google Translate) contains terms related to "presidential", "council", "national", "accord", and "Tripoli."

In [61]:
# Show key words and example posts for a given topic in model #1
def topic_examples(i,n=10000,num_topics=1):
    top_ids = ldamat1[:,i].argsort()[-num_topics:][::-1]
    return(long_posts[['Contents']].iloc[top_ids])
    #print(long_posts.Contents[np.argmax(ldamat1[:,i])][:n])

i = 4
te = topic_examples(i,num_topics=5)
te

,Contents
69,لنتفق الآن\n\nhttps://www.facebook.com/GNAMedia/photos/a.1677608715829850.1073741829.1673617552895633/1717773858480002/?type=3\n\nالمكتب الإعلامي لرئيس المجلس الرئاسي لحكومة الوفاق الوطني\n\nرئيس المجلس الرئاسي السيد فائز السراج يستقبل وزير الشؤون المغاربية والاتحاد الافريقي والجامعة العربية عبد القادر مساهل\n\n{ قبل قليل }\n\nرئيس المجلس الرئاسي السيد (فائز السراج) يستقبل وزير الشؤون المغاربية والاتحاد الافريقي والجامعة العربية (عبد القادر مساهل)
2015,لنتفق الآن\n\nhttps://www.facebook.com/GNAMedia/photos/a.1677608715829850.1073741829.1673617552895633/1717773858480002/?type=3\n\nالمكتب الإعلامي لرئيس المجلس الرئاسي لحكومة الوفاق الوطني\n\nرئيس المجلس الرئاسي السيد فائز السراج يستقبل وزير الشؤون المغاربية والاتحاد الافريقي والجامعة العربية عبد القادر مساهل\n\n{ قبل قليل }\n\nرئيس المجلس الرئاسي السيد (فائز السراج) يستقبل وزير الشؤون المغاربية والاتحاد الافريقي والجامعة العربية (عبد القادر مساهل)
279,لنتفق الآن\n\nhttps://www.facebook.com/GNAMedia/photos/a.1677608715829850.1073741829.1673617552895633/1720813481509373/?type=3\n\nالمكتب الإعلامي لرئيس المجلس الرئاسي لحكومة الوفاق الوطني\n\nرئيس المجلس الرئاسي السيد فائز السراج يستقبل وزير خارجية اسبانيا السيد خوسيه مانويل غارثيا\n\n{ متابعة }\n\nصور لرئيس مجلس الرئاسي لحكومة الوفاق الوطني في استقبال وزير خارجية الاسباني اليوم \n\n#لنتفق_الآن
947,لنتفق الآن\n\n{ متابعة }\n\nهيكلية المقترح الجديد لحكومة الوفاق الوطني كالتالي :\n- وزارة الخارجية \n- وزارة التعاون الدولي\n- وزارة الشؤون العربية والافريقية\nستصبح (وزارة الخارجية والتعاون الدولى)\n- وزارة التربية والتعليم\n- وزارة التعليم والبحث العلمى\n ستصبح (وزارة التعليم والبحت العلمى)\n- وزارة الطيران والنقل الجوى\n- وزارة المواصلات\n- وزارة الاتصالات\nستصبح (وزارة المواصلات والنقل الجوى)\n- وزارة الاقتصاد\n- وزارة التخطيط\n- وزارة المالية\nستصبح (وزارة المالية والتخطيط)\n- وزارة الحكم المحلى\n- وزارة المصالحة الوطنية\n- وزير الدولة لحقوق الانسان\n ستصبح (وزارة الشؤون الاجتماعية والحكم المحلى)\n- وزارة الزراعة\n- وزارة الصناعة .\nستصبح (وزارة الزراعة والصناعة)\n- وزارة الموارد المائية\nستصبح (الهيئة العامة للموارد المائية)\n- وزارة الثقافة\n- وزارة الاعلام\nستصبح (وزارة الاعلام والثقافة)\n- وزارة الكهرباء\nستصبح (المؤسسة العامة للكهرباء)\n- وزارة العمل\n- وزارة التدريب والتكوين المهنى\nستصبح (وزارة العمل والتدريب)\n- وزارة الاسكان\nستصبح (وزارة الاسكان والمرافق)\n- وزارة ال...
1555,لنتفق الآن\n\nhttps://www.facebook.com/GNAMedia/photos/a.1677608715829850.1073741829.1673617552895633/1716479268609461/?type=3\n\nالمكتب الإعلامي لرئيس المجلس الرئاسي لحكومة الوفاق الوطني\n\n{ متابعة }\n\nاجتمع نائب رئيس المجلس الرئاسي لحكومة الوفاق الوطني الدكتور (فتحي المجبري) أول أمس الجمعة الموافق 15 ابريل 2016 مع عدد من عمداء البلديات بالمنطقة الشرقية.\n\n#لنتفق_الآن


In [ ]:
#TODO: A more sophisticated way to do this would be to average the membership probabilities for the two versions of
# topics that match -- the results would (probably) be more robust than either model alone. Small detail.